# **Flight Schedule Optimization**

**Description:**

This Python script optimizes a flight schedule for a fleet of aircraft (tails) over several days. It uses geographic data and a linear regression model to estimate flight times between airports, taking into account the aircraft category and route specifics. The script then applies Google OR-Tools to find an optimal set of flight routes that satisfy constraints on maximum flying time, maximum crew duty time, and minimum required crew rest periods. The optimization objective is to minimize the total duty time for all tails.

The total duty time includes:
1. **Daily Duty Time**: The time between the first departure and last arrival for each tail on each day.
2. **End-of-Day Reposition Time**: The repositioning time required to move the aircraft from the last arrival of one day to the first departure of the next day.

The solution is analyzed and printed, showing the optimized schedules and associated time windows for each flight, as well as the total minimized duty times.

**Author:** Joe Mintz

**Creation Date:** September 3, 2024  
**Last Updated:** September 15, 2024

**Versioning:**

- **Version 1.0 (September 3, 2024)**: Initial release. The objective was to minimize the total route time by reducing the difference between the last arrival time and the first departure time for each aircraft, over the entire schedule.
- **Version 2.0 (September 15, 2024)**: Updated the objective to minimize the total duty time. The new objective includes both daily duty time (time between the first departure and last arrival for each tail per day) and end-of-day reposition time (the time required to move the aircraft from the last arrival of one day to the first departure of the next day).

**Usage:**
To run the script, set the parameters in the `params` dictionary and execute the script. The script will generate an example schedule, optimize it, and print the results.

**References:**
- [Google OR-Tools documentation](https://developers.google.com/optimization)


## Libraries

In [ ]:
from google.colab import drive
drive.mount('drive')

!pip install ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import pandas as pd
import numpy as np
import re
import random
import copy
import csv
import timeit
from math import radians, degrees, atan2, cos, sin, asin, sqrt, log
from datetime import datetime, timedelta
from dateutil import parser

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


## params

In [ ]:
# gen_new_schedule = True: generate a new schedule based on the params, and save it
# gen_new_schedule = False: use the existing saved schedule
gen_new_schedule = True

# The middle date around which the schedule is based
date_input = "2024-09-15"

# The number of days on each side of the date_input
num_days_bound = 2

# Total number of aircraft on the schedule over num_days
num_tails = 5

# Category of the aircraft ('Light Jet', 'Mid Size Jet', 'Super Mid Jet', 'Heavy Jet')
ac_cat = 'Mid Size Jet'

# Maximum flying time per day (standard = 10 hours)
max_fly = 10   # hours

# Maximum crew duty time per day (standard = 14 hours)
max_duty = 14   # hours

# Minimum crew break time between days (standard = 10 hours)
min_break = 10   # hours

# Required turn time after every arrival (standard = 60 minutes)
turn_time = 60   # minutes

# Relax the scheduled time windows by +/- this amount
sch_relax = 15   # minutes

# The initial schedule always adheres to the above max_fly, max_duty, and min_break.
# Those same max_fly, max_duty, and min_break will be used in the optimization constraints UNLESS, below, they are specified to be different for the optimization,
# e.g. the schedule will initially adhere to the above constraints, but can be optimized with tighter constraints below.
# For example, using the dictionary below, incrementally decrease max_fly and max_duty, and increase min_break,
# and observe how the resulting fly, duty, and break times from the optimization result approach these limits as they are tightened.
# Otherwise, set the values in overwr_constr to be None or to be the same as the initially scheduled constraints (in minutes for greater flexibility!).
overwr_constr =  {
                    'max_fly_overwr': None,   # minutes!
                    'max_duty_overwr': None,   # minutes!
                    'min_break_overwr': None   # minutes!
                    }

params =    {
            'gen_new_schedule': gen_new_schedule,
            'date_input': date_input,
            'num_days_bound': num_days_bound,
            'num_tails' : num_tails,
            'ac_cat': ac_cat,
            'max_fly': max_fly,
            'max_duty': max_duty,
            'min_break': min_break,
            'turn_time': turn_time,
            'sch_relax': sch_relax,
            }

## Functions

### flight time functions

In [ ]:
###########################################################################################################################################################

# Calculate haversine distance
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 3956   # radius of earth in miles
    return c * r

# Calculate bearing relative to east
def east_bear(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    x = cos(lat2) * sin(dlon)
    y = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)
    bearingrad = atan2(x, y)
    bearingdeg = degrees(bearingrad)
    eastbearing = (bearingdeg + 270) if (bearingdeg < -90) else (bearingdeg - 90)
    return eastbearing

# Parameters from R linear regression...
# (Intercept)                     -144.0839076
# log(distance)                     81.7968044
# abs(east_bearing)                  0.5338127
# aircraft_categoryLight Jet       -18.3988725
# aircraft_categoryMid Size Jet     -7.1337299
# aircraft_categorySuper Mid Jet    13.2576010
# log(distance):abs(east_bearing)   -0.1363162
# RMSPE for this model: 11.6 minutes, Median absolute error: 5.1 minutes
coefficients = np.array([-144.0839, 81.7968, 0.5338, -18.3989, -7.1337, 13.2576, -0.1363])
# indexes for aircraft category in the regression
catindex = {'Light Jet': 3, 'Mid Size Jet': 4, 'Super Mid Jet': 5, 'Heavy Jet': 0}

# Use the linear regression to predict flight time in minutes based on distance and bearing
def predict_flight_time(lon1, lat1, lon2, lat2, category):
    distance_mi = haversine(lon1, lat1, lon2, lat2)
    if distance_mi == 0:
        return 0
    ln_distance_mi = log(distance_mi)
    abs_bearing_east = abs(east_bear(lon1, lat1, lon2, lat2))
    features = np.array([1, ln_distance_mi, abs_bearing_east, 0, 0, 0, ln_distance_mi * abs_bearing_east])
    features[catindex[category]] = 1   # set the category intercept
    mph_hat = np.dot(features, coefficients)
    return int(60 * distance_mi / mph_hat)

### create_test_schedule

In [ ]:
###########################################################################################################################################################

# This function creates a test schedule to be optimized

def create_test_schedule(params):

    # The middle date around which the schedule is based
    date_input = params['date_input']

    # The number of days on each side of the date_input
    num_days_bound = params['num_days_bound']
    # Get the total number of days
    num_days = 2 * num_days_bound + 1

    # Get the corresponding start date (day 0)
    start_date = datetime.strptime(date_input, "%Y-%m-%d") - timedelta(days = num_days_bound)

    # Total number of aircraft on the schedule over num_days
    num_tails = params['num_tails']

    # Category of the aircraft ('Light Jet', 'Mid Size Jet', 'Super Mid Jet', 'Heavy Jet')
    ac_cat = params['ac_cat']

    # Maximum flying time per day
    max_fly = params['max_fly'] * 60   # minutes

    # Maximum crew duty time per day
    max_duty = params['max_duty'] * 60   # minutes

    # Minimum crew break time between days
    min_break = params['min_break'] * 60   # minutes

    # Required turn time after every arrival
    turn_time = params['turn_time']

    # Relax the scheduled time windows by +/- this amount
    sch_relax = params['sch_relax']

    # Import a csv file of airports with coordinates
    fdir = "/content/drive/MyDrive/flight_schedule_optimization/"
    with open(fdir + "airports.csv", mode='r', newline='') as csvfile:
        csvreader = csv.DictReader(csvfile)
        airports = [row for row in csvreader]
    for airport in airports:
        airport['lat'] = float(airport['lat'])
        airport['lon'] = float(airport['lon'])

    # Helper function to define the number of legs to ADD to the tail each day.
    def get_add_legs(num_days):
        add_legs = []
        for day in range(num_days):
            if day == 0:
                add_legs.append(random.randint(2, 3))   # put 1 or 2 legs on day 0, put 1 leg on day 1
            elif day < num_days - 1:
                add_legs.append(random.randint(1, 2))   # add 0 or 1 legs onto the current day, put 1 leg on the next day
            else:   # the last day
                add_legs.append(random.randint(0, 1))   # add 0 or 1 legs onto the last day
        return add_legs

    # Helper function to, for a given day, set a tail's slack times and set a random start.
    def get_start_and_slack(day, rte):
        if ((day < num_days - 1) and (len(rte) == 2)) or ((day == num_days - 1) and (len(rte) == 1)):   # only 1 leg on the current day
            random_start = random.randint(7*60, 21*60) + 1440 * day   # random time between 7:00am and 9:00pm ET
            slack_times = [random.randint(1, 600) for x in range(len(rte) - 1)]
        else:   # if there is more than 1 leg on the current day, force less slack and an earlier start
            random_start = random.randint(7*60, 12*60) + 1440 * day   # random time between 7:00am and 12:00pm ET
            slack_times = [random.randint(1, 300) for x in range(len(rte) - 1)]
        return random_start, slack_times

    # Helper function to list the flight times corresponding to a list of airport dictionary pairs having ap, lat, and lon
    def list_flight_times(rte, ac_cat):
        rte_flat = [x for sublist in rte for x in sublist]
        flight_times = []
        for i in range(len(rte_flat) - 1):
            leg_dep = rte_flat[i]
            leg_arr = rte_flat[i + 1]
            flight_time = predict_flight_time(leg_dep['lon'], leg_dep['lat'], leg_arr['lon'], leg_arr['lat'], ac_cat)
            flight_times.append(flight_time)
        return flight_times

    # Initialize the schedule
    schedule = []

    # Initialize lists, indexed by tail, to store fly, duty, and break times
    fly_times = [[] for x in range(num_tails)]
    duty_times = [[] for x in range(num_tails)]
    break_times = [[] for x in range(num_tails)]

    # Generate the schedule for each tail and day
    for tail in range(num_tails):

        # Define the number of legs to ADD to the tail each day as add_legs.
        add_legs = get_add_legs(num_days)

        # For each day, store the end of the duty day in a list.
        eodays = []

        for day in range(num_days):

            while True:

                if day == 0:
                    # Initialize a list for the route to store legs and repos, including the end-of-day repo and next-day leg.
                    # After each day where (day < num_days - 1), this list will be cleared out except for the next-day leg, and then repopulated.
                    rte = []
                else:
                    rte = copy.deepcopy(rte_next_day)
                    rte[0][1] = random.choice(airports)   # reselect the arrival airport
                    while rte[0][0]['ap'] == rte[0][1]['ap']:
                        rte[0][1] = random.choice(airports)
                for x in range(add_legs[day]):
                    leg_dep = random.choice(airports)
                    leg_arr = random.choice(airports)
                    while leg_dep['ap'] == leg_arr['ap']:
                        leg_arr = random.choice(airports)
                    rte.append([leg_dep, leg_arr])

                # Set the tail's slack times and set a random start.
                start_and_slack = get_start_and_slack(day, rte)
                random_start = start_and_slack[0]
                slack_times = start_and_slack[1]

                # Get the flight times (both passenger and repo) and turn times.
                flight_times = list_flight_times(rte, ac_cat)
                turn_times = [turn_time if x != 0 else 0 for x in flight_times]

                if day < num_days - 1:
                    # Get the first dep on the current day
                    if day == 0:
                        first_dep = random_start
                    else:   # consider min_break
                        eod = eodays[-1]   # reset eod to the end of the previous duty day
                        first_dep = max((eod + min_break), random_start)
                        # Get the overnight break time
                        break_time = first_dep - eod
                    # Get fly_time and duty_time, but do not include the passenger flight on the next day, hence the :-1,
                    # DO include the REPO to the next day.
                    fly_time = sum(flight_times[0:-1])
                    duty_time = fly_time + sum(turn_times[0:-1]) + sum(slack_times[0:-1])
                    # Get the last dep on the current day.
                    # Essentially it is first_dep + duty_time - (the last passenger flight and repo of the current day).
                    # If there is only one leg on the current day, all of the below additions are zero (first_dep = last_dep).
                    last_dep = first_dep \
                                    + sum(flight_times[0:-3]) \
                                    + sum(turn_times[0:-3]) \
                                    + sum(slack_times[0:-1])
                    # Get the last arr on the current day.
                    last_arr = last_dep + sum(flight_times[-3:-2])
                    # Get the end-of-day repo time
                    eod_repo_time = sum(flight_times[-2:-1])
                    # Get the end of the duty day
                    eod = first_dep + duty_time

                else:   # the last day
                    eod = eodays[-1]   # reset eod to the end of the previous duty day
                    # Get the first dep on the current day
                    first_dep = max((eod + min_break), random_start)
                    # Get the overnight break time
                    break_time = first_dep - eod
                    # Get fly_time and duty_time
                    fly_time = sum(flight_times)
                    duty_time = fly_time + sum(turn_times) + sum(slack_times)
                    # Get the last dep on the current day.
                    # Essentially it is first_dep + duty_time - (the last passenger flight of the current day).
                    # If there is only one leg on the current day, all of the below additions are zero (first_dep = last_dep).
                    last_dep = first_dep \
                                    + sum(flight_times[0:-1]) \
                                    + sum(turn_times[0:-1]) \
                                    + sum(slack_times)
                    # Get the last arr on the current day.
                    last_arr = last_dep + sum(flight_times[-1:])
                    # Get the end-of-day repo time (zero in this case)
                    eod_repo_time = 0
                    # Get the end of the duty day
                    eod = first_dep + duty_time

                cond = (fly_time > max_fly) \
                            or (duty_time > max_duty) \
                            or ((last_dep + sch_relax) >= (1440 * (day + 1)))   # 0, 1440, 2880... corresponds to 12:00am

                if cond:
                    continue

                else:
                    if day < num_days - 1:
                        rte_day = rte[0:-1]   # excluding the next-day passenger flight
                        rte_next_day = rte[-1:]   # before looping back, keep only the last leg in rte because it is for the next day
                        flight_times = flight_times[0:-1]   # excluding the next-day passenger flight
                        turn_times = turn_times[0:-1]   # excluding the turn time after the next-day passenger flight
                        slack_times = slack_times[0:-1]   # excluding the slack time before the next-day passenger flight
                    else:
                        rte_day = copy.deepcopy(rte)

                    flight_times_pass = [flight_times[i] for i in range(0, len(flight_times), 2)]
                    flight_times_repo = [flight_times[i] for i in range(1, len(flight_times), 2)] + [0]
                    turn_times_pass = [turn_times[i] for i in range(0, len(turn_times), 2)]
                    turn_times_repo = [turn_times[i] for i in range(1, len(turn_times), 2)] + [0]
                    slack_times += [0]
                    dep = first_dep
                    eodays.append(eod)   # store the end of the duty day
                    for idx, leg in enumerate(rte_day):
                        # print("Creating leg on tail {}, day {}".format(tail, day))
                        arr = dep + flight_times_pass[idx]
                        dep_dt = start_date + timedelta(minutes=dep)
                        arr_dt = start_date + timedelta(minutes=arr)
                        schedule.append(
                                        {
                                        'tail': tail,
                                        'day': day,
                                        'dep_node': None,
                                        'arr_node': None,
                                        'dep_ap': leg[0]['ap'],
                                        'arr_ap': leg[1]['ap'],
                                        'dep_dt': dep_dt.strftime("%Y-%m-%d %H:%M"),
                                        'arr_dt': arr_dt.strftime("%Y-%m-%d %H:%M"),
                                        'dep': dep,
                                        'dep_min': dep - sch_relax,
                                        'dep_max': dep + sch_relax,
                                        'arr': arr,
                                        'arr_min': arr + turn_time - sch_relax,
                                        'arr_max': arr + turn_time + sch_relax,
                                        'dep_lat': leg[0]['lat'],
                                        'dep_lon': leg[0]['lon'],
                                        'arr_lat': leg[1]['lat'],
                                        'arr_lon': leg[1]['lon'],
                                        'trip_type': 'flight'
                                        }
                                        )
                        dep = arr + turn_times_pass[idx] + slack_times[idx] + flight_times_repo[idx] + turn_times_repo[idx]
                    # Populate the lists to store fly, duty, and break time
                    fly_times[tail].append(fly_time)
                    duty_times[tail].append(duty_time)
                    if day > 0:
                        break_times[tail].append(break_time)
                    break

        # For the tail, arbitrarily append 1440 minutes of break time for the end of the last day
        break_times[tail].append(1440)

    # Store data to be used in the optimization
    opt_data = {}
    opt_data['date_input'] = date_input
    opt_data['num_days_bound'] = num_days_bound
    opt_data['num_days'] = num_days
    opt_data['start_date'] = start_date
    opt_data['num_tails'] = num_tails
    opt_data['ac_cat'] = ac_cat
    opt_data['max_fly'] = max_fly
    opt_data['max_duty'] = max_duty
    opt_data['min_break'] = min_break
    opt_data['turn_time'] = turn_time
    opt_data['sch_relax'] = sch_relax
    opt_data['fly_times'] = fly_times
    opt_data['duty_times'] = duty_times
    opt_data['break_times'] = break_times

    return schedule, opt_data

### create_nodes

In [ ]:
###########################################################################################################################################################

# This function prepares the schedule for the optimization by creating nodes, etc.,

def create_nodes(schedule, opt_data):

    # Extract opt_data to use
    num_days = opt_data['num_days']
    start_date = opt_data['start_date']
    num_tails = opt_data['num_tails']

    # Define a node number for each origin and destination and keep track of the airport nodes in list-of-lists
    schedule = sorted(schedule, key=lambda x: x['dep'])   # the node numbering is ordered by departure time
    dep_nodes = [[] for x in range(num_days)]   # indexed by day number only
    arr_nodes = [[] for x in range(num_days)]   # indexed by day number only
    init_routes = [[[] for x in range(num_days)] for y in range(num_tails)]   # initial routes, indexed by tail and day number
    for idx, leg in enumerate(schedule):
        tail = leg['tail']
        day = leg['day']
        trip_type = 'flight'
        dep_node = idx + 1   # dep_node numbering starts after the depot (0)
        arr_node = dep_node + len(schedule)   # arr_node numbering starts after all dep_nodes
        leg['dep_node'] = dep_node
        leg['arr_node'] = arr_node
        dep_nodes[day].append(dep_node)
        arr_nodes[day].append(arr_node)
        init_routes[tail][day].extend([dep_node, arr_node])

    # The total number of nodes,
    # which is double the number of real flight legs to capture all dep and arr nodes + 1 node for the depot
    num_nodes = 2 * len(schedule) + 1

    # Mark the depot node with a "leg" (mainly for debugging)
    leg = copy.deepcopy(schedule[0])   # arbitrarily copy a leg to edit
    leg['tail'] = None
    leg['day'] = None
    leg['dep_node'] = leg['arr_node'] = 0
    leg['dep_ap'] = leg['arr_ap'] = 'depot'
    leg['dep_dt'] = leg['arr_dt'] = start_date
    leg['dep'] = None
    leg['dep_min'] = 0   # 12:00am on day 0
    # Put dep_max at 11:59pm on the last day or just beyond the max(arr_max), whichever is greater.
    # Note, max(arr_max) could go beyond the last day.
    leg['dep_max'] = max((num_days * 1440 - 1), (max([x['arr_max'] for x in schedule if x['trip_type'] == 'flight']) + 1))
    leg['arr'] = leg['arr_min'] = leg['arr_max'] = None
    leg['dep_lat'] = leg['dep_lon'] = leg['arr_lat'] = leg['arr_lon'] = 0
    leg['trip_type'] = 'depot'
    schedule = schedule + [leg]

    # For some of the above list-of-lists, save flattened lists.
    dep_nodes_flat = sorted([x for sublist in dep_nodes for x in sublist])
    arr_nodes_flat = sorted([x for sublist in arr_nodes for x in sublist])

    # For the optimizer, the initial solution must be a list-of-lists indexed by tail number (excluding the day number index).
    init_solution = copy.deepcopy(init_routes)
    for tail, rte in enumerate(init_solution):
        init_solution[tail] = [x for sublist in rte for x in sublist]

    # Put the leg dep/arr pairs in a list.
    dep_arr = [[dep_nodes_flat[i], arr_nodes_flat[i]] for i in range(len(dep_nodes_flat))]

    # Create a map of node vs. airport for later use
    node_ap_map = {}
    for node in dep_nodes_flat:
        ap = [x['dep_ap'] for x in schedule if x['dep_node'] == node][0]
        node_ap_map[node] = ap
    for node in arr_nodes_flat:
        ap = [x['arr_ap'] for x in schedule if x['arr_node'] == node][0]
        node_ap_map[node] = ap

    # Create a map of node vs. day number for later use
    node_day_map = {}
    for node in range(1, num_nodes):
        day = [x['day'] for x in schedule if x['dep_node'] == node or x['arr_node'] == node][0]
        node_day_map[node] = day

    # Create maps of airport node vs. datetime and time in minutes for later use
    node_dt_map = {}
    node_t_map = {}
    for node in dep_nodes_flat:
        dt = [x['dep_dt'] for x in schedule if x['dep_node'] == node][0]
        t = [x['dep'] for x in schedule if x['dep_node'] == node][0]
        node_dt_map[node] = dt
        node_t_map[node] = t
    for node in arr_nodes_flat:
        dt = [x['arr_dt'] for x in schedule if x['arr_node'] == node][0]
        t = [x['arr'] for x in schedule if x['arr_node'] == node][0]
        node_dt_map[node] = dt
        node_t_map[node] = t

    # Store data to be used in the optimization
    opt_data['num_nodes'] = num_nodes
    opt_data['dep_arr'] = dep_arr   # list-of-lists arbitrarily indexed
    opt_data['dep_nodes'] = dep_nodes   # list-of-lists indexed by day number
    opt_data['arr_nodes'] = arr_nodes   # list-of-lists indexed by day number
    opt_data['dep_nodes_flat'] = dep_nodes_flat   # list
    opt_data['arr_nodes_flat'] = arr_nodes_flat   # list
    opt_data['init_routes'] = init_routes   # list-of-lists indexed by tail and day number
    opt_data['init_solution'] = init_solution   # list-of-lists indexed by tail
    opt_data['node_ap_map'] = node_ap_map   # dict
    opt_data['node_day_map'] = node_day_map   # dict
    opt_data['node_dt_map'] = node_dt_map   # dict
    opt_data['node_t_map'] = node_t_map   # dict

    return schedule, opt_data

### prep_times

In [ ]:
###########################################################################################################################################################

# This function prepares the schedule for the optimization by creating the flight time matrix, time windows, etc.,

def prep_times(schedule, opt_data):

    # Extract opt_data to use
    num_days = opt_data['num_days']
    ac_cat = opt_data['ac_cat']
    num_nodes = opt_data['num_nodes']
    turn_time = opt_data['turn_time']
    dep_nodes = opt_data['dep_nodes']
    arr_nodes = opt_data['arr_nodes']
    dep_nodes_flat = opt_data['dep_nodes_flat']
    arr_nodes_flat = opt_data['arr_nodes_flat']
    node_t_map = opt_data['node_t_map']

    # Get the lats and lons to compute the flight time matrix,
    # ensuring that looping happens according to node order!!
    airport_coords = []
    for dep_node in dep_nodes_flat:
        airport_coords.append([[float(x['dep_lat']), float(x['dep_lon'])] for x in schedule if x['dep_node'] == dep_node][0])
    for arr_node in arr_nodes_flat:
        airport_coords.append([[float(x['arr_lat']), float(x['arr_lon'])] for x in schedule if x['arr_node'] == arr_node][0])

    # Function to generate the flight time matrix
    def make_time_matrix(airport_coords, aircraft_category):
        coords_array = np.array(airport_coords)
        time_mat = np.zeros((len(coords_array), len(coords_array)), dtype=int)
        for i, a in enumerate(coords_array):
            for j, b in enumerate(coords_array):
                time_mat[i, j] = predict_flight_time(a[1], a[0], b[1], b[0], aircraft_category)
        return time_mat

    # Now construct the flight time matrix using make_time_matrix
    time_mat_fly = make_time_matrix(airport_coords, ac_cat)

    # Insert the depot into the time matrix
    depot_row = np.zeros((1, len(time_mat_fly) + 1), dtype=int)
    time_mat_fly = np.insert(time_mat_fly, 0, 0, axis=1)
    time_mat_fly = np.vstack([depot_row, time_mat_fly])
    time_mat_fly = time_mat_fly.tolist()   # convert to a list-of-lists
    time_mat_fly = [[int(element) for element in row] for row in time_mat_fly]   # convert each element from numpy.int64 to python int

    # The above time_mat_fly is used in the optimization to constrain fly time.
    # Now generate a second time matrix to constrain total time.
    # This one includes turn_time after every flight arrival. Add turn_time to all non-zero elements in the matrix.
    time_mat = np.array(time_mat_fly)
    time_mat = np.where(time_mat != 0, time_mat + turn_time, time_mat)
    time_mat = time_mat.tolist()   # convert to a list-of-lists
    time_mat = [[int(element) for element in row] for row in time_mat]   # convert each element from numpy.int64 to python int

    # Create a map of elapsed time between dep_node and arr_node when they are on the same day and when arr > dep
    # Essentially this matrix captures potential duty times for the day, excluding the end-of-day repo.
    # It will be used in the optimization objective.
    day_duty_mat = np.zeros((num_nodes, num_nodes), dtype=int)
    for day in range(num_days):
        for dep_node in dep_nodes[day]:
            dep = node_t_map[dep_node]
            for arr_node in arr_nodes[day]:
                arr = node_t_map[arr_node]
                if arr > dep:
                    day_duty_mat[dep_node][arr_node] = arr - dep
    day_duty_mat = day_duty_mat.tolist()   # convert to a list-of-lists
    day_duty_mat = [[int(element) for element in row] for row in day_duty_mat]   # convert each element from numpy.int64 to python int

    # Now that no further adjustments to time windows are to be made, put the time windows into a list.
    time_windows = []   # to be a list of tuples
    for node in range(num_nodes):
        leg_idx = [idx for idx, x in enumerate(schedule) if x['dep_node'] == node or x['arr_node'] == node][0]   # index of the leg in the schedule
        if node == schedule[leg_idx]['dep_node']:
            tw_min = schedule[leg_idx]['dep_min']
            tw_max = schedule[leg_idx]['dep_max']
        else:
            tw_min = schedule[leg_idx]['arr_min']
            tw_max = schedule[leg_idx]['arr_max']
        time_windows.append((tw_min, tw_max))

    # Store data to be used in the optimization
    opt_data['time_mat'] = time_mat   # list-of-lists indexed by node
    opt_data['time_mat_fly'] = time_mat_fly   # list-of-lists indexed by node
    opt_data['day_duty_mat'] = day_duty_mat   # list-of-lists indexed by node
    opt_data['time_windows'] = time_windows   # list-of-tuples indexed by node

    return schedule, opt_data

### run_opt

In [ ]:
###########################################################################################################################################################

# This function uses Google OR-Tools to define the constraints and run the optimization.
# It will find an optimal set of flight routes that satisfy the constraints, with the objective of minimizing the total duty time for all tails.
# The total duty time includes:
# 1. Daily Duty Time: The time between the first departure and last arrival for each tail on each day.
# 2. End-of-Day Reposition Time: The repositioning time required to move the aircraft from the last arrival of one day to the first departure of the next day.

def run_opt(opt_data, overwr_constr):

    start_time_f = timeit.default_timer()

    # Extract opt_data to use
    num_days = opt_data['num_days']
    num_tails = opt_data['num_tails']
    max_fly = opt_data['max_fly']
    max_duty = opt_data['max_duty']
    min_break= opt_data['min_break']
    num_nodes = opt_data['num_nodes']
    dep_arr = opt_data['dep_arr']
    dep_nodes = opt_data['dep_nodes']
    arr_nodes = opt_data['arr_nodes']
    init_solution = opt_data['init_solution']
    node_day_map = opt_data['node_day_map']
    time_mat = opt_data['time_mat']
    time_mat_fly = opt_data['time_mat_fly']
    day_duty_mat = opt_data['day_duty_mat']
    time_windows = opt_data['time_windows']

    # If overwr_constr is populated, use it for the optimization constraints,
    # which may be different from the constraints to which the initial schedule adheres.
    if overwr_constr['max_fly_overwr'] is not None:
        max_fly = overwr_constr['max_fly_overwr']
    if overwr_constr['max_duty_overwr'] is not None:
        max_duty = overwr_constr['max_duty_overwr']
    if overwr_constr['min_break_overwr'] is not None:
        min_break = overwr_constr['min_break_overwr']

    depot_node = 0
    max_wait = num_days * 1440
    max_tail_time = (num_days + 1) * 1440   # the ((last arr on last day) - (first dep on first day)) for a tail will never exceed max_tail_time

    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(num_nodes, num_tails, depot_node)

    # Create routing model
    routing = pywrapcp.RoutingModel(manager)

    #################################################################################################################################################

    # Define the callbacks and dimensions for
    # flight_time + turn_time, based on time_mat
    # and flight_time only, based on time_mat_fly

    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return time_mat[from_node][to_node]

    def fly_time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return time_mat_fly[from_node][to_node]

    # Register their callbacks
    time_callback_index = routing.RegisterTransitCallback(time_callback)
    fly_time_callback_index = routing.RegisterTransitCallback(fly_time_callback)

    dimension_name = 'Time'
    routing.AddDimension(
        time_callback_index,
        max_wait,  # allow waiting time
        max_tail_time,  # maximum time per tail
        False,  # don't force start cumul to zero.
        dimension_name)
    time_dimension = routing.GetDimensionOrDie(dimension_name)

    dimension_name = 'Fly_Time'
    routing.AddDimension(
        fly_time_callback_index,
        0,  # no waiting time
        max_tail_time,  # maximum time per tail
        False,  # don't force start cumul to zero.
        dimension_name)
    fly_time_dimension = routing.GetDimensionOrDie(dimension_name)

    #################################################################################################################################################

    #################################################################################################################################################
    ################################ --------------------- THE OBJECTIVE !!! --------------------- ##################################################
    #################################################################################################################################################

    # The objective is to minimize the sum of duty time over all days and tails.

    # First, define a callback to capture the day duty times, excluding the end-of-day repos.
    def day_duty_time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)

        # If the depot is involved, return zero cost
        if (from_node == depot_node) or (to_node == depot_node):
            return 0

        else:
            from_day = node_day_map[from_node]
            to_day = node_day_map[to_node]

            # day_duty_time is for same-day dep to arr, thus return zero if not the same day.
            if from_day != to_day:
                return 0

            else:
                # Collect all indices of the nodes for the day before and the day after the current day.
                if from_day == 0:   # if the current day is day 0, use the starting index of the tail
                    prev_day_indices = [routing.Start(tail) for tail in range(num_tails)]
                else:
                    prev_day_indices = [manager.NodeToIndex(node) for node in dep_nodes[from_day-1]]
                if to_day == num_days - 1:   # if the current day is the last day, use the ending index of the tail
                    next_day_indices = [routing.End(tail) for tail in range(num_tails)]
                else:
                    next_day_indices = [manager.NodeToIndex(node) for node in arr_nodes[to_day+1]]

                # Conditions to identify the tail's bounding airport indices for the current day.
                from_index_starts_day = False
                to_index_ends_day = False
                for prev_day_index in prev_day_indices:
                    if (routing.NextVar(prev_day_index) == from_index) and (routing.VehicleVar(prev_day_index) == routing.VehicleVar(from_index)):
                        from_index_starts_day = True   # means that from_index is the starting index of the current day
                for next_day_index in next_day_indices:
                    if (routing.NextVar(to_index) == next_day_index) and (routing.VehicleVar(to_index) == routing.VehicleVar(next_day_index)):
                        to_index_ends_day = True   # means that to_index is the ending index of the current day

                # Full conditional expression: Ensure same tail, and that from and to indices start and end the day
                if (routing.VehicleVar(from_index) == routing.VehicleVar(to_index)) and from_index_starts_day and to_index_ends_day:
                    return day_duty_mat[from_node][to_node]
                else:
                    return 0

    # Next, define a callback to capture the end-of-day repo times.
    def eod_repo_time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)

        # If the depot is involved, return zero cost
        if (from_node == depot_node) or (to_node == depot_node):
            return 0

        else:
            from_day = node_day_map[from_node]
            to_day = node_day_map[to_node]

            # eod_repo_time is the tail's reposition time from the last arr of the current day to the first dep of the next day,
            # thus return zero if to_day != from_day + 1
            if to_day != from_day + 1:
                return 0

            else:
                # Full conditional expression: Ensure same tail, and that to_index is right after from_index
                if (routing.NextVar(from_index) == to_index) and (routing.VehicleVar(from_index) == routing.VehicleVar(to_index)):
                    return time_mat[from_node][to_node]   # use time_mat to get the flight time plus turn time
                else:
                    return 0

    # Finally, define a callback to capture the total duty times (day_duty_time + eod_repo_time)
    def total_duty_time_callback(from_index, to_index):
        day_duty_time = day_duty_time_callback(from_index, to_index)
        eod_repo_time = eod_repo_time_callback(from_index, to_index)
        return day_duty_time + eod_repo_time

    # Register the callback
    total_duty_time_callback_index = routing.RegisterTransitCallback(total_duty_time_callback)

    # Define the cost of each arc
    routing.SetArcCostEvaluatorOfAllVehicles(total_duty_time_callback_index)

    #################################################################################################################################################
    #################################################################################################################################################
    #################################################################################################################################################

    ###########################################-------------------------------------------------------------------###############################
    ##### TIME WINDOW CONSTRAINTS --------------------------------------------------------------------------------###############################

    # Add time window constraints for each location except depot
    for node, tw in enumerate(time_windows):
        if node == depot_node:
            continue
        index = manager.NodeToIndex(node)
        time_dimension.CumulVar(index).SetRange(tw[0], tw[1])

    # Add time window constraints for each tail start node
    for tail in range(num_tails):
        index = routing.Start(tail)
        time_dimension.CumulVar(index).SetRange(time_windows[depot_node][0], time_windows[depot_node][1])

    ###########################################-------------------------------------------------------------------###############################
    ##### LEG DEPARTURE TO ARRIVAL CONSTRAINTS -------------------------------------------------------------------###############################

    # Define the tail constraints for all legs
    for leg in dep_arr:
        dep_node = leg[0]
        arr_node = leg[1]
        routing.solver().Add(routing.NextVar(manager.NodeToIndex(dep_node)) == manager.NodeToIndex(arr_node))   # arr must be immediately after dep

    #################################################################################################################################################
    ################################# --------------------- CREW CONSTRAINTS --------------------- ##################################################
    #################################################################################################################################################

    ###########################################-------------------------------------------------------------------###############################
    ##### FLY AND DUTY TIME CONSTRAINTS --------------------------------------------------------------------------###############################

    # The total fly_time per tail per day, including the end-of-day reposition, must be <= max_fly

    # The total duty_time per tail per day is the elapsed time (last arr + turn_time - first dep) plus the fly_time (+ turn_time) of the end-of-day reposition.
    # The total duty_time per tail per day must be <= max_duty

    # Since there might not always be an end-of-day repo...
    # First, constrain between the first departure node and last arrival node of the day.
    # But since first dep and last arr of the day are unknown, constrain between all deps and arrs for the day when dep and arr are on the same tail.
    for day in range(num_days):
        for dep_node in dep_nodes[day]:
            dep_index = manager.NodeToIndex(dep_node)
            for arr_node in arr_nodes[day]:
                arr_index = manager.NodeToIndex(arr_node)
                fly_time = fly_time_dimension.CumulVar(arr_index) - fly_time_dimension.CumulVar(dep_index)
                duty_time = time_dimension.CumulVar(arr_index) - time_dimension.CumulVar(dep_index)
                # The decision variable
                node_condition = (routing.VehicleVar(dep_index) == routing.VehicleVar(arr_index))
                # The constraints to add if the decision variable is true
                fly_constr_cond = (fly_time <= max_fly)
                duty_constr_cond = (duty_time <= max_duty)
                # If the decision variable is false, then the constraint never applies, but instead the value of 1 is applied (always true).
                fly_expression = routing.solver().ConditionalExpression(node_condition, fly_constr_cond, 1)
                duty_expression = routing.solver().ConditionalExpression(node_condition, duty_constr_cond, 1)
                routing.solver().AddConstraint(fly_expression >= 1)
                routing.solver().AddConstraint(duty_expression >= 1)

    # Now, account for situations where there is an end-of-day repo.
    for day in range(num_days - 1):
        for dep_node in dep_nodes[day]:
            dep_index = manager.NodeToIndex(dep_node)
            for arr_node in arr_nodes[day]:
                arr_index = manager.NodeToIndex(arr_node)
                # Note, getting fly_time_dimension.CumulVar between arr_index and next_dep_index (see below) won't get the eod_repo
                # for total fly_time and total duty_time because the time to and from dummy nodes (e.g. dum_node_b) is zero.
                # Therefore, the workaround is to first get the fly and duty_time for the day
                day_fly_time = sum([(routing.VehicleVar(dep_index) == routing.VehicleVar(arr_index)) \
                                * (fly_time_dimension.CumulVar(arr_index) - fly_time_dimension.CumulVar(dep_index))])
                day_duty_time = sum([(routing.VehicleVar(dep_index) == routing.VehicleVar(arr_index)) \
                                * (time_dimension.CumulVar(arr_index) - time_dimension.CumulVar(dep_index))])
                for next_dep_node in dep_nodes[day+1]:
                    next_dep_index = manager.NodeToIndex(next_dep_node)
                    # The decision variable
                    node_condition = (routing.NextVar(arr_index) == next_dep_index \
                                        and routing.VehicleVar(arr_index) == routing.VehicleVar(next_dep_index))
                    # Note above in node_condition, routing.VehicleVar(arr_index) == routing.VehicleVar(next_dep_index) seems redundant,
                    # but it is necessary!
                    # Now constrain day_fly_time to max_fly with the eod_repo (+ turn_time) backed out
                    # and day_duty_time to max_duty with the eod_repo (+ turn_time) backed out
                    max_day_fly_time = max_fly - time_mat_fly[arr_node][next_dep_node]   # use time_mat_fly to get only the fly_time
                    max_day_duty_time = max_duty - time_mat[arr_node][next_dep_node]   # use time_mat to get the fly_time + turn_time
                    # The constraints to add if the decision variable is true
                    fly_constr_cond = (day_fly_time <= max_day_fly_time)
                    duty_constr_cond = (day_duty_time <= max_day_duty_time)
                    # If the decision variable is false, then the constraint never applies, but instead the value of 1 is applied (always true).
                    fly_expression = routing.solver().ConditionalExpression(node_condition, fly_constr_cond, 1)
                    duty_expression = routing.solver().ConditionalExpression(node_condition, duty_constr_cond, 1)
                    routing.solver().AddConstraint(fly_expression >= 1)
                    routing.solver().AddConstraint(duty_expression >= 1)

    ###########################################-------------------------------------------------------------------###############################
    ##### BREAK CONSTRAINTS --------------------------------------------------------------------------------------###############################

    # The break_time starts after the end-of-day reposition (+ turn_time) and ends at the first departure of the next day.
    # The total break_time per tail per day must be >= min_break

    for day in range(num_days - 1):
        for arr_node in arr_nodes[day]:
            arr_index = manager.NodeToIndex(arr_node)
            for next_dep_node in dep_nodes[day+1]:
                next_dep_index = manager.NodeToIndex(next_dep_node)
                # Get the overnight elapsed time, which covers the eod_repo + turn_time + break_time
                repo_and_break = time_dimension.CumulVar(next_dep_index) - time_dimension.CumulVar(arr_index)
                # The decision variable
                node_condition = (routing.NextVar(arr_index) == next_dep_index)
                # Now constrain repo_and_break to min_break with the eod_repo (+ turn_time) added
                min_repo_and_break = min_break + time_mat[arr_node][next_dep_node]   # use time_mat to get the fly_time + turn_time
                # The constraints to add if the decision variable is true
                break_constr_cond = (repo_and_break >= min_repo_and_break)
                # If the decision variable is false, then the constraint never applies, but instead the value of 1 is applied (always true).
                break_expression = routing.solver().ConditionalExpression(node_condition, break_constr_cond, 1)
                routing.solver().AddConstraint(break_expression >= 1)

    #################################################################################################################################################
    #################################################################################################################################################
    #################################################################################################################################################

    # Instantiate route start and end times to produce feasible times
    for tail in range(num_tails):
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(tail)))
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(tail)))

    #################################################################################################################################################

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()

    search_parameters.time_limit.seconds = 30
    init_solution = routing.ReadAssignmentFromRoutes(init_solution, True)
    solution = routing.SolveFromAssignmentWithParameters(init_solution, search_parameters)

    elapsed = round((timeit.default_timer() - start_time_f), 5)
    print("\nCompleted run_opt in {} s".format(elapsed))

    return manager, routing, solution

### get_opt_routes

In [ ]:
###########################################################################################################################################################

# This function constructs the optimized routes of nodes from the solution

def get_opt_routes(opt_data, opt_output):

    # Extract opt_data to use
    num_days = opt_data['num_days']
    num_tails = opt_data['num_tails']
    node_day_map = opt_data['node_day_map']

    manager, routing, solution = opt_output[0], opt_output[1], opt_output[2]

    opt_routes = []
    for tail in range(num_tails):
        # Split each route by day
        rte = [[] for x in range(num_days)]
        index = solution.Value(routing.NextVar(routing.Start(tail)))
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            if node > 0:   # exclude the depot
                day = node_day_map[node]
                rte[day].append(node)
            index = solution.Value(routing.NextVar(index))
        opt_routes.append(rte)

    return opt_routes

### calc_crew_times

In [ ]:
###########################################################################################################################################################

# This function post-processes the solution to calculate fly, duty, and break time for each tail and day

def calc_crew_times(opt_data, opt_output, opt_routes):

    # Extract opt_data to use
    num_days = opt_data['num_days']
    num_tails = opt_data['num_tails']
    min_break = params['min_break']
    time_mat = opt_data['time_mat']
    time_mat_fly = opt_data['time_mat_fly']

    manager, routing, solution = opt_output[0], opt_output[1], opt_output[2]
    time_dimension = routing.GetDimensionOrDie('Time')

    ############################################################################################################################################

    # Initialize lists, indexed by tail, to store fly, duty, and break times
    fly_times = [[] for x in range(num_tails)]
    duty_times = [[] for x in range(num_tails)]
    break_times = [[] for x in range(num_tails)]

    for tail in range(num_tails):

        for day in range(num_days):

            rte = opt_routes[tail][day]

            if day < num_days - 1:
                next_day_rte = opt_routes[tail][day+1]
                all_next_days_rte = [x for sublist in opt_routes[tail][day+1:] for x in sublist]
            else:
                next_day_rte = []
                all_next_days_rte = []

            if len(rte) > 0:

                # Get the FLY and DUTY time
                # The fly time is the sum of all flight times over the day, including all repo flight times of the day, including the end-of-day repo
                flight_times = []
                for idx in range(len(rte) - 1):
                    # Use time_mat_fly to get only FLY time, not the extra turn_time
                    flight_times.append(time_mat_fly[rte[idx]][rte[idx+1]])
                fly_time = sum(flight_times)
                # The DUTY time is ([the last arr time of the day] - [the first dep time of the day] + turn_time + [the end-of-day repo] + turn_time)
                last_arr = solution.Min(time_dimension.CumulVar(manager.NodeToIndex(rte[-1])))   # the min of the last arr (includes turn time)
                first_dep = solution.Min(time_dimension.CumulVar(manager.NodeToIndex(rte[0])))   # the min of the first dep
                duty_time = last_arr - first_dep
                if len(next_day_rte) > 0:   # add the end-of-day repo time to the fly and duty time
                    fly_time += time_mat_fly[rte[-1]][next_day_rte[0]]   # use time_mat_fly to get only the fly time of the end-of-day repo
                    duty_time += time_mat[rte[-1]][next_day_rte[0]]   # use time_mat to get the fly time + turn time of the end-of-day repo

                # Get the BREAK time
                if len(all_next_days_rte) > 0:
                    # Consider the next day that has a flight (not necessarily day+1)
                    break_start = last_arr + time_mat[rte[-1]][all_next_days_rte[0]]   # use time_mat to get the fly time + turn time of the end-of-day repo
                    break_end = solution.Min(time_dimension.CumulVar(manager.NodeToIndex(all_next_days_rte[0])))   # the min of the next dep
                    break_time = break_end - break_start
                else:
                    break_time = 1440   # arbitrarily record 1440 minutes of break time when none of the following days have flights

            else:
                fly_time = 0
                duty_time = 0
                break_time = 1440   # arbitrarily record 1440 minutes of break time when the current day has no flights

            fly_times[tail].append(fly_time)
            duty_times[tail].append(duty_time)
            break_times[tail].append(break_time)

    return fly_times, duty_times, break_times

### print functions

In [ ]:
###########################################################################################################################################################

# This function prints the solution from the optimization with TIME WINDOWS,
# OR it will print the initial solution with time windows

def print_time_window_solution(opt_data, opt_output, opt_routes):

    # Extract opt_data to use
    start_date = opt_data['start_date']
    ac_cat = opt_data['ac_cat']
    ac_cat_upper = ac_cat.upper()
    init_routes = opt_data['init_routes']
    node_ap_map = opt_data['node_ap_map']
    time_windows = opt_data['time_windows']

    if opt_output and opt_routes:
        routes = copy.deepcopy(opt_routes)
        manager, routing, solution = opt_output[0], opt_output[1], opt_output[2]
        time_dimension = routing.GetDimensionOrDie('Time')
        print("######## Optimized Solution ########")
        print("\nObjective: {}\n".format(solution.ObjectiveValue()))
    else:
        routes = copy.deepcopy(init_routes)
        print("######## Initial Solution ########\n")

    total_time = 0

    for tail, rte in enumerate(routes):
        plan_output = "{} ({}):".format(ac_cat_upper, tail)

        for day, day_rte in enumerate(rte):
            date = str(datetime.strptime(start_date.date().strftime("%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S") \
                            + timedelta(days = day)).split(' ')[0]
            plan_output += "\n{}: ".format(date)
            for node in day_rte:
                ap = node_ap_map[node]
                if opt_output and opt_routes:
                    index = manager.NodeToIndex(node)
                    time_var = time_dimension.CumulVar(index)
                    rte_time = solution.Max(time_var)   # really only needed for the last node
                    plan_output += "{} ({}) Time({},{}) -> ".format(ap, node, solution.Min(time_var), solution.Max(time_var))
                else:
                    rte_time = time_windows[node][1]   # really only needed for the last node
                    plan_output += "{} ({}) Time({},{}) -> ".format(ap, node, time_windows[node][0], time_windows[node][1])
            if len(day_rte) > 0:
                plan_output = ' '.join(plan_output.split(' ')[:-2])   # remove the last ->

        plan_output += "\nTime of the route: {} min\n".format(rte_time)
        total_time += rte_time

        print(plan_output)

    print("Total time of all routes: {} min".format(total_time))

In [ ]:
###########################################################################################################################################################

# This function prints the solution from the optimization with TIME POINTS, based on the time windows,
# OR it will print the initial solution with the initially scheduled dep and arr times.

def print_time_point_solution(schedule, opt_data, opt_output, opt_routes):

    # Extract opt_data to use
    start_date = opt_data['start_date']
    ac_cat = opt_data['ac_cat']
    ac_cat_upper = ac_cat.upper()
    arr_nodes_flat = opt_data['arr_nodes_flat']
    init_routes = opt_data['init_routes']
    node_ap_map = opt_data['node_ap_map']
    node_dt_map = opt_data['node_dt_map']
    time_mat_fly = opt_data['time_mat_fly']
    time_windows = opt_data['time_windows']

    # Helper function to convert the optimization result minutes back to ET time
    def convert_optminutes_to_et(start_date, minutes):
        datetime_str_et = str(start_date + timedelta(minutes=minutes))
        datetime_str_et = re.split('\\+', datetime_str_et)[0]
        time_str_et = ' '.join(datetime_str_et.split(' ')[-1:])
        time_str_et = ':'.join(time_str_et.split(':')[:-1]) + ' ET'
        return time_str_et

    if opt_output and opt_routes:
        routes = copy.deepcopy(opt_routes)
        manager, routing, solution = opt_output[0], opt_output[1], opt_output[2]
        time_dimension = routing.GetDimensionOrDie('Time')
        print("######## Optimized Solution ########\n")
    else:
        routes = copy.deepcopy(init_routes)
        print("######## Initial Solution ########\n")

    for tail, rte in enumerate(routes):
        plan_output = "{} ({}):".format(ac_cat_upper, tail)

        for day, day_rte in enumerate(rte):
            date = str(datetime.strptime(start_date.date().strftime("%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S") \
                            + timedelta(days = day)).split(' ')[0]
            plan_output += "\n{}: ".format(date)

            for node in day_rte:
                if opt_output and opt_routes:
                    if node in arr_nodes_flat:
                        dep_node = day_rte[day_rte.index(node) - 1]
                        arr_node = node
                        time_var = time_dimension.CumulVar(manager.NodeToIndex(dep_node))
                        t = int((solution.Min(time_var) + solution.Max(time_var)) / 2) + time_mat_fly[dep_node][arr_node]
                    else:
                        time_var = time_dimension.CumulVar(manager.NodeToIndex(node))
                        t = int((solution.Min(time_var) + solution.Max(time_var)) / 2)
                    time_str_et = convert_optminutes_to_et(start_date, t)
                else:
                    time_str_et = node_dt_map[node]
                    time_str_et = ' '.join(time_str_et.split(' ')[-1:]) + ' ET'   # remove the date part for the initial solution
                ap = node_ap_map[node]
                plan_output += "{} ({}) ({}) -> ".format(ap, node, time_str_et)

            plan_output = ' '.join(plan_output.split(' ')[:-2])   # remove the last ->

        plan_output += "\n"
        print(plan_output)

### run all

In [ ]:
if params['gen_new_schedule']:

    # Based on params, generate a new example schedule to be optimized
    schedule, opt_data = create_test_schedule(params)

    fdir = "/content/drive/MyDrive/flight_schedule_optimization/"

    # Write the schedule to a csv
    with open(fdir + "schedule.csv", mode='w', newline='') as csvfile:
        csvwriter = csv.DictWriter(csvfile, fieldnames=schedule[0].keys())
        csvwriter.writeheader()
        csvwriter.writerows(schedule)

    # Write opt_data to a csv
    with open(fdir + "opt_data.csv", mode='w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['key', 'value'])
        for key, value in opt_data.items():
            csvwriter.writerow([key, repr(value)])   # use repr to avoid eval issues

In [ ]:
# For the schedule.csv import, define a function to convert string values back to their original types
def convert_types(row):
    converted_row = {}
    for key, value in row.items():
        if value == '':
            converted_row[key] = None   # convert empty strings back to None
        elif value.isdigit():
            converted_row[key] = int(value)   # convert digit strings to integers
        else:
            try:
                converted_row[key] = float(value)   # attempt to convert to float
            except ValueError:
                converted_row[key] = value   # keep as string if not a float
    return converted_row

fdir = "/content/drive/MyDrive/flight_schedule_optimization/"

# Read schedule.csv back into a list of dictionaries with correct types
with open(fdir + "schedule.csv", mode='r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    schedule = [convert_types(row) for row in csvreader]

# Read opt_data.csv back into a dictionary with deserialization
with open(fdir + "opt_data.csv", mode='r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)   # skip the header
    opt_data = {}
    for rows in csvreader:
        key, value = rows[0], rows[1]
        try:
            if "datetime" in value:   # detect datetime-like strings
                value = value.replace("datetime.datetime", "datetime")   # adjust the string for eval
                value = eval(value, {"datetime": datetime, "timedelta": timedelta})
            elif isinstance(value, str) and "T" in value:
                value = parser.isoparse(value)   # convert ISO string back to datetime
            else:
                value = eval(value)   # evaluate for other types
        except Exception as e:
            print(f"Error evaluating {value}: {e}")
        opt_data[key] = value

schedule, opt_data = create_nodes(schedule, opt_data)
schedule, opt_data = prep_times(schedule, opt_data)

print(opt_data.keys())
sch_fly_times, sch_duty_times, sch_break_times = opt_data['fly_times'], opt_data['duty_times'], opt_data['break_times']
# Unpack the opt_data dictionary and set all other variables
locals().update(opt_data)

opt_output = run_opt(opt_data, overwr_constr)
manager, routing, solution = opt_output[0], opt_output[1], opt_output[2]

dict_keys(['date_input', 'num_days_bound', 'num_days', 'start_date', 'num_tails', 'ac_cat', 'max_fly', 'max_duty', 'min_break', 'turn_time', 'sch_relax', 'fly_times', 'duty_times', 'break_times', 'num_nodes', 'dep_arr', 'dep_nodes', 'arr_nodes', 'dep_nodes_flat', 'arr_nodes_flat', 'init_routes', 'init_solution', 'node_ap_map', 'node_day_map', 'node_dt_map', 'node_t_map', 'time_mat', 'time_mat_fly', 'day_duty_mat', 'time_windows'])

Completed run_opt in 6.44172 s


In [ ]:
# Debugging
# schedule_df = pd.DataFrame(schedule)
# schedule_df = schedule_df.sort_values(by=['dep_node']).reset_index(drop=True)
# schedule_df.head()

## Solution

In [ ]:
if solution:
    print("\nSolution found.")
    opt_routes = get_opt_routes(opt_data, opt_output)
    opt_fly_times, opt_duty_times, opt_break_times = calc_crew_times(opt_data, opt_output, opt_routes)
else:
    print("\nNo solution found.")


Solution found.


In [ ]:
# Print initial solution with time windows
print_time_window_solution(opt_data, None, None)

######## Initial Solution ########

MID SIZE JET (0):
2024-09-13: KHAB (4) Time(693,723) -> KNIP (44) Time(833,863) -> KPOB (7) Time(975,1005) -> KMKG (47) Time(1154,1184)
2024-09-14: KEWB (11) Time(1933,1963) -> KTQH (51) Time(2216,2246) -> KCEV (13) Time(2461,2491) -> KCJR (53) Time(2592,2622)
2024-09-15: KJQF (23) Time(3846,3876) -> KSEP (63) Time(4084,4114)
2024-09-16: KUBS (26) Time(4838,4868) -> KSLH (66) Time(5037,5067) -> KMHL (32) Time(5394,5424) -> KPLD (72) Time(5531,5561)
2024-09-17: KBIV (38) Time(6529,6559) -> KHSB (78) Time(6661,6691)
Time of the route: 6691 min

MID SIZE JET (1):
2024-09-13: KJYG (3) Time(609,639) -> KVPC (43) Time(803,833) -> KDEC (9) Time(1059,1089) -> KCHK (49) Time(1230,1260)
2024-09-14: KVUJ (14) Time(2466,2496) -> KSPS (54) Time(2708,2738)
2024-09-15: KLSF (18) Time(3488,3518) -> KMGE (58) Time(3577,3607) -> KTMA (22) Time(3743,3773) -> KGCN (62) Time(4067,4097)
2024-09-16: KEFD (30) Time(5341,5371) -> KMMU (70) Time(5598,5628)
2024-09-17: KLFI (3

In [ ]:
if solution:
    # Print optimized solution with time windows
    print_time_window_solution(opt_data, opt_output, opt_routes)

######## Optimized Solution ########

Objective: 8438

MID SIZE JET (0):
2024-09-13: KOQU (2) Time(451,481) -> KCMD (42) Time(678,708) -> KSUX (6) Time(879,909) -> KOLZ (46) Time(985,1015)
2024-09-14: KBWW (10) Time(1846,1876) -> KGPI (50) Time(2009,2039)
2024-09-15: KPBH (17) Time(3324,3333) -> KFFX (57) Time(3437,3446) -> KJQF (23) Time(3846,3855) -> KSEP (63) Time(4084,4093)
2024-09-16: KFTW (29) Time(5332,5362) -> KUUU (69) Time(5605,5635)
2024-09-17: KLFI (36) Time(6454,6454) -> KHNZ (76) Time(6548,6548)
Time of the route: 6548 min

MID SIZE JET (1):
2024-09-13: KMMH (8) Time(986,1016) -> KLCI (48) Time(1359,1389)
2024-09-14: KVUJ (14) Time(2466,2496) -> KSPS (54) Time(2708,2738)
2024-09-15: KLSF (18) Time(3488,3518) -> KMGE (58) Time(3577,3607) -> KTMA (22) Time(3743,3773) -> KGCN (62) Time(4067,4097)
2024-09-16: KATY (33) Time(5532,5532) -> KFWB (73) Time(5695,5695)
2024-09-17: KAEJ (37) Time(6485,6485) -> KARM (77) Time(6680,6680)
Time of the route: 6680 min

MID SIZE JET (2):


In [ ]:
# Print initial solution with time points
print_time_point_solution(schedule, opt_data, None, None)

######## Initial Solution ########

MID SIZE JET (0):
2024-09-13: KHAB (4) (11:48 ET) -> KNIP (44) (13:08 ET) -> KPOB (7) (16:30 ET) -> KMKG (47) (18:29 ET)
2024-09-14: KEWB (11) (08:28 ET) -> KTQH (51) (12:11 ET) -> KCEV (13) (17:16 ET) -> KCJR (53) (18:27 ET)
2024-09-15: KJQF (23) (16:21 ET) -> KSEP (63) (19:19 ET)
2024-09-16: KUBS (26) (08:53 ET) -> KSLH (66) (11:12 ET) -> KMHL (32) (18:09 ET) -> KPLD (72) (19:26 ET)
2024-09-17: KBIV (38) (13:04 ET) -> KHSB (78) (14:16 ET)

MID SIZE JET (1):
2024-09-13: KJYG (3) (10:24 ET) -> KVPC (43) (12:38 ET) -> KDEC (9) (17:54 ET) -> KCHK (49) (19:45 ET)
2024-09-14: KVUJ (14) (17:21 ET) -> KSPS (54) (20:23 ET)
2024-09-15: KLSF (18) (10:23 ET) -> KMGE (58) (10:52 ET) -> KTMA (22) (14:38 ET) -> KGCN (62) (19:02 ET)
2024-09-16: KEFD (30) (17:16 ET) -> KMMU (70) (20:33 ET)
2024-09-17: KLFI (36) (11:49 ET) -> KHNZ (76) (12:23 ET)

MID SIZE JET (2):
2024-09-13: KOQU (2) (07:46 ET) -> KCMD (42) (10:33 ET) -> KSUX (6) (14:54 ET) -> KOLZ (46) (15:40 ET)

In [ ]:
if solution:
    # Print optimized solution with time points
    print_time_point_solution(schedule, opt_data, opt_output, opt_routes)

######## Optimized Solution ########

MID SIZE JET (0):
2024-09-13: KOQU (2) (07:46 ET) -> KCMD (42) (10:33 ET) -> KSUX (6) (14:54 ET) -> KOLZ (46) (15:40 ET)
2024-09-14: KBWW (10) (07:01 ET) -> KGPI (50) (08:44 ET)
2024-09-15: KPBH (17) (07:28 ET) -> KFFX (57) (08:21 ET) -> KJQF (23) (16:10 ET) -> KSEP (63) (19:08 ET)
2024-09-16: KFTW (29) (17:07 ET) -> KUUU (69) (20:40 ET)
2024-09-17: KLFI (36) (11:34 ET) -> KHNZ (76) (12:08 ET)

MID SIZE JET (1):
2024-09-13: KMMH (8) (16:41 ET) -> KLCI (48) (21:54 ET)
2024-09-14: KVUJ (14) (17:21 ET) -> KSPS (54) (20:23 ET)
2024-09-15: KLSF (18) (10:23 ET) -> KMGE (58) (10:52 ET) -> KTMA (22) (14:38 ET) -> KGCN (62) (19:02 ET)
2024-09-16: KATY (33) (20:12 ET) -> KFWB (73) (21:55 ET)
2024-09-17: KAEJ (37) (12:05 ET) -> KARM (77) (14:20 ET)

MID SIZE JET (2):
2024-09-13: KJYG (3) (10:24 ET) -> KVPC (43) (12:38 ET) -> KPOB (7) (16:30 ET) -> KMKG (47) (18:29 ET)
2024-09-14: KGGW (12) (11:09 ET) -> KLEE (52) (15:20 ET) -> KGYB (16) (21:38 ET) -> KMQY (56

In [ ]:
print("scheduled max_fly = {}".format(max_fly))

print("\n### Scheduled fly times ###")
for tail in range(num_tails):
    print(sch_fly_times[tail])

print("\n\noptimized max_fly = {}".format(overwr_constr['max_fly_overwr'] if overwr_constr['max_fly_overwr'] is not None else max_fly))

if solution:
    print("\n### Optimized fly times ###")
    for tail in range(num_tails):
        print(opt_fly_times[tail])

scheduled max_fly = 600

### Scheduled fly times ###
[389, 450, 272, 371, 72]
[477, 302, 490, 255, 34]
[487, 529, 462, 233, 389]
[456, 263, 225, 463, 378]
[576, 309, 541, 426, 167]


optimized max_fly = 600

### Optimized fly times ###
[459, 263, 356, 294, 34]
[441, 302, 485, 233, 135]
[492, 529, 231, 371, 72]
[353, 472, 473, 372, 387]
[365, 309, 417, 463, 355]


In [ ]:
print("scheduled max_duty = {}".format(max_duty))
print("total scheduled duty time over all tails and days = {}".format(sum(sum(x) for x in sch_duty_times)))

print("\n### Scheduled duty times ###")
for tail in range(num_tails):
    print(sch_duty_times[tail])

print("\n\noptimized max_duty = {}".format(overwr_constr['max_duty_overwr'] if overwr_constr['max_duty_overwr'] is not None else max_duty))
print("total optimized duty time over all tails and days = {}".format(sum(sum(x) for x in opt_duty_times)))

if solution:
    print("\n### Optimized duty times ###")
    for tail in range(num_tails):
        print(opt_duty_times[tail])

scheduled max_duty = 840
total scheduled duty time over all tails and days = 14586

### Scheduled duty times ###
[640, 776, 392, 793, 132]
[827, 422, 795, 375, 94]
[735, 805, 783, 353, 687]
[744, 383, 568, 784, 614]
[696, 429, 802, 730, 227]


optimized max_duty = 840
total optimized duty time over all tails and days = 14481

### Optimized duty times ###
[707, 383, 840, 414, 94]
[561, 422, 790, 353, 195]
[781, 805, 351, 793, 132]
[641, 798, 766, 699, 637]
[638, 429, 665, 784, 803]


In [ ]:
print("scheduled min_break = {}".format(min_break))

print("\n### Scheduled break times ###")
for tail in range(num_tails):
    print(sch_break_times[tail])

print("\n\noptimized min_break = {}".format(overwr_constr['min_break_overwr'] if overwr_constr['min_break_overwr'] is not None else min_break))

if solution:
    print("\n### Optimized break times ###")
    for tail in range(num_tails):
        print(opt_break_times[tail])

scheduled min_break = 600

### Scheduled break times ###
[600, 1137, 600, 898, 1440]
[1030, 600, 1058, 738, 1440]
[908, 600, 1250, 600, 1440]
[657, 1074, 1114, 600, 1440]
[886, 600, 616, 601, 1440]


optimized min_break = 600

### Optimized break times ###
[688, 1095, 1168, 708, 1440]
[919, 600, 1254, 600, 1440]
[704, 600, 988, 898, 1440]
[847, 902, 616, 632, 1440]
[1237, 600, 723, 600, 1440]
